# NB-03 Exec Overview Artifact (Tab 1 Mirror, Exec-Safe)

Purpose: Generate an exec-safe summary from DS0 + DS1 marts only, export markdown and PNG figures, and embed guardrails and QC receipt.

## Scope

- Reads DS0 + DS1 marts only (optional 1-week QC from staging not enabled by default).
- Exports:
  - `docs/executive_summary.md`
  - `docs/images/nb03_trends_grid.png`
- No refactors or model changes.

In [1]:
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from notebooks.utils import story_blocks as sb

plt.style.use("seaborn-v0_8-whitegrid")

def fmt_date(value) -> str:
    if value is None or pd.isna(value):
        return "?"
    return pd.to_datetime(value).strftime("%Y-%m-%d")

def fmt_dollars_k(value: float | None) -> str:
    if value is None or pd.isna(value):
        return "?"
    return f"${value / 1000.0:,.1f}K"

def fmt_rate_pct(value: float | None) -> str:
    if value is None or pd.isna(value):
        return "?"
    return f"{value * 100:,.1f}%"

def fmt_pp(value: float | None) -> str:
    if value is None or pd.isna(value):
        return "?"
    sign = "+" if value > 0 else "-" if value < 0 else ""
    return f"{sign}{abs(value):,.1f}pp"

def fmt_count(value: float | None) -> str:
    if value is None or pd.isna(value):
        return "?"
    return f"{int(round(value)):,}"

def fmt_dollars_k_or_na(value: float | None) -> str:
    if value is None or pd.isna(value) or float(value) == 0:
        return "N/A"
    return fmt_dollars_k(value)

def format_wow(value: float | None, kind: str) -> str:
    if value is None or pd.isna(value):
        return "?"
    sign = "+" if value > 0 else "-" if value < 0 else ""
    abs_val = abs(value)
    if kind == "dollars_k":
        return f"{sign}${abs_val:,.1f}K"
    if kind == "pp":
        return f"{sign}{abs_val:,.1f}pp"
    if kind == "count":
        return f"{sign}{int(round(abs_val)):,}"
    return str(value)



In [2]:
import os

OFFLINE = os.getenv("OFFLINE") == "1"

BQ_PROJECT_ID = os.getenv("BQ_PROJECT_ID") or os.getenv("GOOGLE_CLOUD_PROJECT")
BQ_DATASET_ID = os.getenv("BQ_DATASET_ID")

def load_bq_table(table_name: str):
    if not BQ_PROJECT_ID or not BQ_DATASET_ID:
        return None
    try:
        import pandas_gbq
    except Exception:
        return None
    query = f"SELECT * FROM `{BQ_PROJECT_ID}.{BQ_DATASET_ID}.{table_name}`"
    return pandas_gbq.read_gbq(query, project_id=BQ_PROJECT_ID)

ds0 = sb.load_table(
    "ds0",
    OFFLINE,
    lambda: load_bq_table("mart_exec_overview_latest_week"),
    "docs/fixtures/ds0.csv",
)

ds1 = sb.load_table(
    "ds1",
    OFFLINE,
    lambda: load_bq_table("mart_exec_kpis_weekly_complete"),
    "docs/fixtures/ds1.csv",
)

if ds0 is None or ds1 is None:
    raise RuntimeError("BigQuery not configured. Set BQ_PROJECT_ID and BQ_DATASET_ID.")


In [3]:
import pandas as pd

# Fail-safe checks
if len(ds0) != 1:
    raise ValueError("DS0 expected single-row latest week KPI strip; aborting export.")

ds0_row = ds0.iloc[0]
required_ds1_cols = {
    "week_start",
    "is_complete_week",
    "in_last_52_complete_weeks",
    "latest_complete_week_start",
}
missing_ds1 = required_ds1_cols - set(ds1.columns)
if missing_ds1:
    raise ValueError(f"DS1 missing required columns: {sorted(missing_ds1)}")

ds1_complete = ds1[(ds1["is_complete_week"] == True) & (ds1["in_last_52_complete_weeks"] == True)].copy()
ds1_complete = ds1_complete.sort_values("week_start")

complete_weeks_count = len(ds1_complete)
insufficient_trend_history = complete_weeks_count < 8

latest_complete_week_start = ds1_complete["latest_complete_week_start"].max()
ds0_week_start = ds0_row.get("week_start")
week_match = pd.notna(latest_complete_week_start) and (pd.Timestamp(latest_complete_week_start) == pd.Timestamp(ds0_week_start))

prior_week_start = ds0_row.get("prior_complete_week_start")
prior_week_missing = pd.isna(prior_week_start)

maturity_days = None
if "maturity_days" in ds1_complete.columns and len(ds1_complete) > 0:
    maturity_days = int(ds1_complete.iloc[-1]["maturity_days"])
elif "maturity_days" in ds1.columns and len(ds1) > 0:
    maturity_days = int(ds1.iloc[-1]["maturity_days"])
else:
    maturity_days = 60

# Trend horizon: last 13 complete weeks
trend_df = ds1_complete.sort_values("week_start").tail(13).copy()


In [4]:
# DATA RECEIPT - DS1 PROOF
print('DATA RECEIPT - DS1 PROOF')
print(f'BQ_PROJECT_ID: {BQ_PROJECT_ID}')
print(f'BQ_DATASET_ID: {BQ_DATASET_ID}')
print(f'DS1 total rows loaded: {len(ds1)}')
if 'is_complete_week' in ds1.columns:
    print(f'DS1 rows where is_complete_week = TRUE: {int(ds1["is_complete_week"].sum())}')
else:
    print('DS1 rows where is_complete_week = TRUE: N/A (column missing)')
if 'in_last_52_complete_weeks' in ds1.columns:
    print(f'DS1 rows where in_last_52_complete_weeks = TRUE: {int(ds1["in_last_52_complete_weeks"].sum())}')
else:
    print('DS1 rows where in_last_52_complete_weeks = TRUE: N/A (column missing)')
print(f'DS1 rows after complete-week filtering: {len(ds1_complete)}')
print(f'DS1 rows in last 13-week slice: {len(trend_df)}')
if len(ds1_complete) > 0:
    print(f'week_start min: {ds1_complete["week_start"].min()}')
    print(f'week_start max: {ds1_complete["week_start"].max()}')
else:
    print('week_start min: N/A')
    print('week_start max: N/A')

cols = [
    'week_start',
    'payer_allowed_amt',
    'observed_paid_amt',
    'payer_yield_gap_amt',
    'denial_rate',
    'n_claims',
    'denied_potential_allowed_proxy_amt',
]
cols = [c for c in cols if c in ds1_complete.columns]
print('HEAD (3)')
print(ds1_complete[cols].head(3).to_string(index=False) if cols else 'N/A')
print('TAIL (3)')
print(ds1_complete[cols].tail(3).to_string(index=False) if cols else 'N/A')


DATA RECEIPT - DS1 PROOF
BQ_PROJECT_ID: None
BQ_DATASET_ID: None
DS1 total rows loaded: 12
DS1 rows where is_complete_week = TRUE: 12
DS1 rows where in_last_52_complete_weeks = TRUE: 12
DS1 rows after complete-week filtering: 12
DS1 rows in last 13-week slice: 12
week_start min: 2010-10-04
week_start max: 2010-12-20
HEAD (3)
week_start  payer_allowed_amt  observed_paid_amt  payer_yield_gap_amt  denial_rate  n_claims  denied_potential_allowed_proxy_amt
2010-10-04           700000.0           760000.0              20000.0        0.128      8000                             40000.0
2010-10-11           701000.0           748000.0              20100.0        0.129      8010                             40200.0
2010-10-18           702000.0           742000.0              20200.0        0.130      8020                             40400.0
TAIL (3)
week_start  payer_allowed_amt  observed_paid_amt  payer_yield_gap_amt  denial_rate  n_claims  denied_potential_allowed_proxy_amt
2010-12-06         

In [5]:
# Build KPI strip table (latest complete week)
recoupment_all_zero = False
if 'recoupment_amt' in ds1_complete.columns and len(ds1_complete) > 0:
    rec_series = ds1_complete['recoupment_amt'].fillna(0)
    recoupment_all_zero = bool((rec_series == 0).all())

kpis = [
    {
        'label': 'Payer Yield Gap',
        'col': 'payer_yield_gap_amt',
        'wow_col': 'wow_yield_gap_amt_k',
        'value_fmt': fmt_dollars_k,
        'wow_fmt': lambda v: format_wow(v, 'dollars_k'),
        'unit': '$K',
    },
    {
        'label': 'Denied Potential Allowed Proxy',
        'col': 'denied_potential_allowed_proxy_amt',
        'wow_col': 'wow_denied_proxy_amt_k',
        'value_fmt': fmt_dollars_k,
        'wow_fmt': lambda v: format_wow(v, 'dollars_k'),
        'unit': '$K',
    },
    {
        'label': 'Denial Rate',
        'col': 'denial_rate',
        'wow_col': 'wow_denial_rate_pp',
        'value_fmt': fmt_rate_pct,
        'wow_fmt': lambda v: format_wow(v, 'pp'),
        'unit': '%',
    },
    {
        'label': 'Payer Allowed',
        'col': 'payer_allowed_amt',
        'wow_col': 'wow_payer_allowed_amt_k',
        'value_fmt': fmt_dollars_k,
        'wow_fmt': lambda v: format_wow(v, 'dollars_k'),
        'unit': '$K',
    },
    {
        'label': 'Observed Paid',
        'col': 'observed_paid_amt',
        'wow_col': 'wow_observed_paid_amt_k',
        'value_fmt': fmt_dollars_k,
        'wow_fmt': lambda v: format_wow(v, 'dollars_k'),
        'unit': '$K',
    },
    {
        'label': 'Claim Count',
        'col': 'n_claims',
        'wow_col': 'wow_n_claims',
        'value_fmt': fmt_count,
        'wow_fmt': lambda v: format_wow(v, 'count'),
        'unit': 'count',
    },
    {
        'label': 'Recoupment',
        'col': 'recoupment_amt',
        'wow_col': 'wow_recoupment_amt_k',
        'value_fmt': (lambda v: 'N/A' if recoupment_all_zero else fmt_dollars_k(v)),
        'wow_fmt': (lambda v: 'N/A' if recoupment_all_zero else format_wow(v, 'dollars_k')),
        'unit': '' if recoupment_all_zero else '$K',
    },
]

kpi_rows = []
for kpi in kpis:
    value = ds0_row.get(kpi['col']) if kpi['col'] in ds0_row else None
    wow_value = ds0_row.get(kpi['wow_col']) if kpi['wow_col'] else None
    unit = kpi['unit']
    if kpi['label'] == 'Recoupment' and recoupment_all_zero:
        unit = ''
    kpi_rows.append({
        'KPI': kpi['label'],
        'Value': kpi['value_fmt'](value),
        'Unit': unit,
        'WoW': kpi['wow_fmt'](wow_value),
    })

kpi_table = pd.DataFrame(kpi_rows)


In [6]:
# KPI strip image
kpi_strip_rows = kpi_table.copy()
if recoupment_all_zero:
    kpi_strip_rows = kpi_strip_rows[kpi_strip_rows['KPI'] != 'Recoupment']

fig, ax = plt.subplots(figsize=(7, 2.2))
ax.axis('off')
table = ax.table(
    cellText=kpi_strip_rows[['KPI', 'Value', 'WoW']].values.tolist(),
    colLabels=['KPI', 'Value', 'WoW'],
    loc='center'
)
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 1.2)

doc_dir = Path('docs') if Path('docs').is_dir() else Path('..') / 'docs'
img_dir = doc_dir / 'images'
img_dir.mkdir(parents=True, exist_ok=True)
kpi_strip_path = img_dir / 'nb03_kpi_strip.png'
try:
    fig.savefig(kpi_strip_path, dpi=150, bbox_inches='tight')
except Exception as exc:
    plt.close(fig)
    raise RuntimeError(f'Failed to save KPI strip image to {kpi_strip_path}') from exc
plt.close(fig)


In [7]:
# Trend plots (2x3 grid, last 13 complete weeks)
trend_metrics = [
    ("payer_yield_gap_amt", "Yield Gap ($)", True),
    ("denied_potential_allowed_proxy_amt", "Denied Potential Allowed Proxy ($)", True),
    ("denial_rate", "Denial Rate (%)", False),
    ("payer_allowed_amt", "Payer Allowed ($)", True),
    ("observed_paid_amt", "Observed Paid ($)", True),
    ("n_claims", "Claim Count", False),
]

fig, axes = plt.subplots(2, 3, figsize=(14, 7), constrained_layout=True)
axes = axes.flatten()

for ax, (col, title, is_dollars) in zip(axes, trend_metrics):
    series = trend_df[["week_start", col]].copy()
    series = series.sort_values("week_start")
    series["week_start"] = pd.to_datetime(series["week_start"])
    y = series[col].astype(float)
    if is_dollars:
        y_plot = y / 1000.0
        ax.set_ylabel("$K")
    elif col == "denial_rate":
        y_plot = y * 100.0
        ax.set_ylabel("%")
    else:
        y_plot = y
    ax.plot(series["week_start"], y_plot, marker="o", linewidth=1.5, label="Weekly")
    roll = y_plot.rolling(4, min_periods=1).mean()
    ax.plot(series["week_start"], roll, linestyle="--", linewidth=1.2, label="4-week avg")
    if pd.notna(latest_complete_week_start):
        ax.axvline(pd.Timestamp(latest_complete_week_start), color="black", linestyle=":", linewidth=1)
    ax.set_title(title)
    ax.tick_params(axis="x", rotation=45)
    ax.legend(loc="upper left", fontsize=8)

for idx in range(len(trend_metrics), len(axes)):
    axes[idx].axis("off")

doc_dir = Path("docs") if Path("docs").is_dir() else Path("..") / "docs"
img_dir = doc_dir / "images"
img_dir.mkdir(parents=True, exist_ok=True)
trend_path = img_dir / "nb03_trends_grid.png"
fig.savefig(trend_path, dpi=150)
plt.close(fig)


In [8]:
# Executive summary bullets (derived from DS0 values)

def money_change_phrase_k(value_k):
    if value_k is None or pd.isna(value_k):
        return 'N/A'
    v = float(value_k)
    direction = 'up' if v >= 0 else 'down'
    return f"{direction} ${abs(v):,.1f}K"

summary_delta = ds0_row.get('wow_observed_paid_amt_k')
table_delta = ds0_row.get('wow_observed_paid_amt_k')
if pd.notna(summary_delta) and pd.notna(table_delta):
    if abs(float(summary_delta) - float(table_delta)) > 0.1:
        print('WARNING: Summary delta mismatch')
        summary_delta = table_delta

change_bullet = f'WoW change: Observed Paid {money_change_phrase_k(summary_delta)}'

denial_rate = ds0_row.get('denial_rate')
rate_txt = fmt_rate_pct(denial_rate)
rate_pp = ds0_row.get('wow_denial_rate_pp')
rate_pp_txt = format_wow(rate_pp, 'pp') if pd.notna(rate_pp) else '--'
rate_label = 'Stable' if (pd.notna(rate_pp) and abs(float(rate_pp)) < 0.2) else 'Moved'
rate_bullet = f'{rate_label}: Denial rate {rate_txt} (Delta {rate_pp_txt})'

mix_flag = str(ds0_row.get('mix_stability_flag'))
mix_reason = str(ds0_row.get('mix_stability_reason'))
if mix_flag == 'CHECK SEGMENTS':
    watch_bullet = f'Watch: Mix stability flagged - {mix_reason}'
else:
    watch_bullet = 'Watch: Mix stability OK'

summary_bullets = [
    change_bullet,
    rate_bullet,
    watch_bullet,
]


In [9]:
# QC snapshot
ds0_row_count = len(ds0)
ds1_complete_count = complete_weeks_count

qc_rows = [
    ("DS0 row count", str(ds0_row_count)),
    ("DS1 complete-week count (last 52)", str(ds1_complete_count)),
    ("Mix stability", f"{ds0_row.get('mix_stability_flag')} — {ds0_row.get('mix_stability_reason')}"),
    ("Mix threshold", ">15% vs baseline median"),
    ("DS1 latest week matches DS0", str(bool(week_match))),
]

insufficient_banner = ""
if insufficient_trend_history:
    insufficient_banner = "Insufficient complete-week history for stable trending."

prior_missing_banner = ""
if prior_week_missing:
    prior_missing_banner = "Comparator (prior complete week) is missing; WoW deltas shown as —."


In [10]:
# Export markdown artifact
import os

def df_to_markdown(df: pd.DataFrame) -> str:
    headers = list(df.columns)
    rows = df.values.tolist()
    lines = [
        "| " + " | ".join(headers) + " |",
        "| " + " | ".join(["---"] * len(headers)) + " |",
    ]
    for row in rows:
        lines.append("| " + " | ".join(str(value) for value in row) + " |")
    return "\n".join(lines)

def ascii_safe(text: object) -> str:
    if text is None:
        return ""
    s = str(text)
    s = s.replace("—", " - ")
    s = s.replace("Δ", "Delta")
    s = s.replace("⚠", "WARNING:")
    return s

kpi_table_ascii = kpi_table.copy()
if "Value" in kpi_table_ascii.columns:
    kpi_table_ascii["Value"] = kpi_table_ascii["Value"].replace({"?": "N/A", "—": "N/A"})
if "WoW" in kpi_table_ascii.columns:
    kpi_table_ascii["WoW"] = kpi_table_ascii["WoW"].replace({"?": "--", "—": "--"})

kpi_md = df_to_markdown(kpi_table_ascii)

include_generated_on = os.getenv("INCLUDE_GENERATED_ON") == "1"

generated_on = datetime.now().strftime("%Y-%m-%d %H:%M:%S") if include_generated_on else None

history_line = ""
if ds1_complete_count < 52:
    history_line = f"History available: {ds1_complete_count} complete weeks (target 52)."

ds1_min_week = "N/A"
if len(ds1_complete) > 0:
    ds1_min_week = fmt_date(ds1_complete["week_start"].min())

ds1_max_week = "N/A"
if len(ds1_complete) > 0:
    ds1_max_week = fmt_date(ds1_complete["week_start"].max())

summary_bullets_ascii = [ascii_safe(item).replace("?", "N/A") for item in summary_bullets]

# Summary lines (strip prefixes to avoid duplication)
wow_line = summary_bullets_ascii[0].replace("WoW change: ", "") if len(summary_bullets_ascii) > 0 else "N/A"
moved_or_stable_line = summary_bullets_ascii[1].replace("Stable: ", "").replace("Moved: ", "") if len(summary_bullets_ascii) > 1 else "N/A"
watch_line = summary_bullets_ascii[2].replace("Watch: ", "") if len(summary_bullets_ascii) > 2 else "N/A"

as_of_date = fmt_date(ds0_row.get("as_of_date"))
anchor_week = fmt_date(ds0_row.get("week_start"))
comparator_week = fmt_date(ds0_row.get("prior_complete_week_start"))
svc_start = ds1_min_week
svc_end = ds1_max_week

partial_week_lines = []
if bool(ds0_row.get("is_partial_week_present")):
    partial_week_lines = [
        "## Partial Week Banner",
        f"WARNING: Partial-week activity detected (Start: {fmt_date(ds0_row.get('partial_week_start'))}, Claims: {fmt_count(ds0_row.get('partial_week_n_claims'))}).",
        "Exec KPIs and trends reflect complete + mature weeks only. Partial-week is excluded from KPI comparisons.",
    ]

recoupment_note = ""
if recoupment_all_zero:
    recoupment_note = "Recoupment is N/A because it is NULL/0 across the complete-week window."

mix_flag = ascii_safe(ds0_row.get("mix_stability_flag"))
mix_reason = ascii_safe(ds0_row.get("mix_stability_reason"))

qc_lines = [f"- {label}: {value}" for label, value in qc_rows]
qc_lines = [ascii_safe(line).replace("?", "N/A") for line in qc_lines]

insufficient_banner = ascii_safe(insufficient_banner).replace("?", "N/A") if insufficient_banner else ""
prior_missing_banner = ascii_safe(prior_missing_banner).replace("?", "N/A") if prior_missing_banner else ""

lines = [
    "# Executive Summary (Latest Complete + Mature Week)",
    f"- WoW change: {wow_line}",
    f"- {moved_or_stable_line}",
    f"- Watch: {watch_line}",
    "## Data Stamp (Receipt)",
    f"Model as_of_date (from marts): {as_of_date}",
    f"Anchor week: {anchor_week}",
    f"Comparator: {comparator_week}",
    "Included weeks: complete-week only (DS1)",
    f"Service timeline (complete weeks): {svc_start} to {svc_end}",
    f"Maturity: {maturity_days}",
    f"Mix stability: {mix_flag} - {mix_reason}",
]

if history_line:
    lines.append(history_line)
if include_generated_on:
    lines.append(f"Generated on: {generated_on}")

lines += [
    "",
    *partial_week_lines,
    "",
    "## KPI Strip + WoW Deltas",
    "![KPI Strip](images/nb03_kpi_strip.png)",
    "Units: $K (1 decimal) for dollars; % (1 decimal) for rates; pp (1 decimal) for denial WoW; counts with commas.",
    "Comparator: prior complete week",
    kpi_md,
    recoupment_note,
    "",
    "## Trends (Last 13 Complete Weeks)",
    "Trends include 4-week rolling average to reduce volatility.",
    f"![Trends Grid](images/{trend_path.name})",
    "",
    "## Guardrails & Definitions",
    "- Complete-week logic uses DS1 is_complete_week",
    "- Maturity gating is enforced upstream; report uses mart outputs",
    "- Proxy metric is directional ranking, not guaranteed recovery",
    '- Mix stability threshold: deviation >15% vs baseline median triggers "CHECK SEGMENTS"',
    "",
    "## Data Notes",
    "- Synthetic dataset disclosure (CMS DE-SynPUF).",
    "- No protected health information (PHI).",
    "",
    "## QC Snapshot",
    *qc_lines,
    "",
    insufficient_banner,
    prior_missing_banner,
]

lines = [line for line in lines if line is not None and line != ""]
lines = [ascii_safe(line).replace("?", "N/A") for line in lines]

md_text = "\n".join(lines) + "\n"

doc_dir = Path("docs") if Path("docs").is_dir() else Path("..") / "docs"
doc_dir.mkdir(parents=True, exist_ok=True)
doc_path = doc_dir / "executive_summary.md"
with open(doc_path, "w", encoding="utf-8") as handle:
    handle.write(md_text)

print(f"Wrote {doc_path}")
print(f"Wrote {trend_path}")
print(f"Wrote {kpi_strip_path}")

# Decision memo export (ASCII-only)

def wow_text_dollars_k(value, comparator_available):
    if not comparator_available:
        return "N/A (comparator not available)"
    if value is None or pd.isna(value):
        return "N/A (comparator not available)"
    v = float(value)
    direction = "up" if v >= 0 else "down"
    return f"{direction} ${abs(v):,.1f}K"

def wow_text_pp(value, comparator_available):
    if not comparator_available:
        return "N/A (comparator not available)"
    if value is None or pd.isna(value):
        return "N/A (comparator not available)"
    v = float(value)
    sign = "+" if v >= 0 else "-"
    return f"{sign}{abs(v):,.1f}pp"

def level_text_rate(value):
    if value is None or pd.isna(value):
        return "N/A"
    return fmt_rate_pct(value)

comparator_available = pd.notna(ds0_row.get("prior_complete_week_start"))

recommendation_lines = []
if str(ds0_row.get("mix_stability_flag")) == "OK":
    recommendation_lines.append("- If mix stability is OK: proceed to NB-04 (Drivers) to size top contributors, then NB-05 (Workqueue) to allocate capacity.")
else:
    recommendation_lines.append("- If mix stability is CHECK SEGMENTS: investigate volume/segment mix first; defer queue expansion until drivers are validated.")

missing_fields = []
what_moved_lines = []

if "wow_observed_paid_amt_k" in ds0_row:
    what_moved_lines.append(f"- Observed Paid: {wow_text_dollars_k(ds0_row.get('wow_observed_paid_amt_k'), comparator_available)}")
else:
    missing_fields.append("wow_observed_paid_amt_k")

if "denial_rate" in ds0_row and "wow_denial_rate_pp" in ds0_row:
    denial_rate_level = ds0_row.get("denial_rate")
    denial_rate_wow = ds0_row.get("wow_denial_rate_pp")
    if comparator_available and pd.notna(denial_rate_level) and pd.notna(denial_rate_wow):
        what_moved_lines.append(f"- Denial Rate: {level_text_rate(denial_rate_level)} (Delta {wow_text_pp(denial_rate_wow, comparator_available)})")
    else:
        what_moved_lines.append("- Denial Rate: N/A (comparator not available)")
else:
    if "denial_rate" not in ds0_row:
        missing_fields.append("denial_rate")
    if "wow_denial_rate_pp" not in ds0_row:
        missing_fields.append("wow_denial_rate_pp")

if "wow_payer_allowed_amt_k" in ds0_row:
    what_moved_lines.append(f"- Payer Allowed: {wow_text_dollars_k(ds0_row.get('wow_payer_allowed_amt_k'), comparator_available)}")
else:
    missing_fields.append("wow_payer_allowed_amt_k")

n_weeks = len(ds1_complete)
min_week = fmt_date(ds1_complete["week_start"].min()) if n_weeks > 0 else "N/A"
max_week = fmt_date(ds1_complete["week_start"].max()) if n_weeks > 0 else "N/A"

mix_flag = ascii_safe(ds0_row.get("mix_stability_flag"))
mix_reason = ascii_safe(ds0_row.get("mix_stability_reason"))

status_label = "STABLE" if str(ds0_row.get("mix_stability_flag")) == "OK" else "INVESTIGATE"
status_reason = mix_reason if mix_reason else "mix stability flagged"

note_lines = []
if missing_fields:
    note_lines.append("- Note: Some fields not available in marts for this run; memo lines were omitted rather than inferred.")

memo_lines = [
    "# Decision Memo - Exec Overview (Latest Complete + Mature Week)",
    "",
    "## Recommendation (conditional)",
    *recommendation_lines,
    "",
    "## What moved (from DS0; reconciles to KPI table)",
    *what_moved_lines,
    "",
    "## Receipt (trust stamp)",
    f"- Model as_of_date (from marts): {fmt_date(ds0_row.get('as_of_date'))}",
    f"- Anchor week: {fmt_date(ds0_row.get('week_start'))}",
    f"- Comparator: {fmt_date(ds0_row.get('prior_complete_week_start'))}",
    f"- Service timeline (complete weeks): {min_week} to {max_week} ({n_weeks} weeks)",
    "- Included weeks: complete-week only (DS1); mature-only enforced upstream",
    f"- Mix stability: {mix_flag} - {mix_reason}",
]

if include_generated_on:
    memo_lines.append(f"- Generated on: {generated_on}")

memo_lines += [
    "",
    "## Notes / interpretation status",
    f"- Interpretation status: {status_label} - {status_reason}",
    "- Paid vs Allowed differences are claim-file amounts used for directional prioritization, not adjudicated underpayment findings.",
    "- Denied Potential Allowed Proxy is directional prioritization only; not guaranteed recovery.",
    *note_lines,
    "",
    "## Follow-ups (not executed in NB-03)",
    "- NB-04 (Drivers): quantify top contributors (contribution/composition; not causal claims).",
    "- NB-05 (Workqueue): demonstrate prioritization using proxy metrics with explicit no-recovery guarantee language.",
]

memo_lines = [ascii_safe(line) for line in memo_lines]

memo_path = doc_dir / "decision_memo_latest_complete_week.md"
with open(memo_path, "w", encoding="utf-8") as handle:
    handle.write("\n".join(memo_lines))

print(f"Wrote {memo_path}")


Wrote ..\docs\executive_summary.md
Wrote ..\docs\images\nb03_trends_grid.png
Wrote ..\docs\images\nb03_kpi_strip.png
Wrote ..\docs\decision_memo_latest_complete_week.md
